## ENCODE GMT to SIG File Conversion

This script will take an input of an ENCODE (from any year) and transform it into the SIG file format. Make sure to carefully match the ENCODE version you are using to the SIG file version you are outputting.

In [ ]:
%run /home/maayanlab/Desktop/Projects/KEA3/Scripts/init.ipy

In [ ]:
#this maximum set according to the max column value indicated to me by Ariel
colnames = ['']*7001
colnames

ENCODE = pd.read_table('ENCODE_TF_ChIP-seq_2014.txt', names = colnames)

subset = ENCODE.copy()

subset.dropna(how = 'all', inplace = True)

subset.replace(np.nan, '', inplace = True)

subset.head()

In [ ]:
for column in subset:
    for index, row in subset['%s' %column].iteritems():
        s = row
        if ',' in s:
            s = s.split(',', 1)[0]
        subset.loc[index, '%s' %column] = s

subset.head()

In [ ]:
#loop through each row of the data frame
#make the index ('') the dataframe's key
#make each item of rowData into part of a list
#store this list as a value for the index key of the dictionary


#Once the dictionary is completed, we can use the dictionary to drop values which are NaN from the
#list (or can have used list comprehension beforehand to filter the list and 
#keep NaNs from becoming values in the dataframe)

subset.set_index('', inplace = True)

d = {}

for index, rowData in subset.iterrows():
    lst = []
    sr = pd.Series(rowData)
    for i in range(0, len(sr)):
        if sr[i] == '':
            lst = lst + [i]
    sr.drop(sr.index[lst], inplace = True)
    d[index] = [sr.values]

d

In [ ]:
#Create an empty dataframe to house entries from the dictionary
#for each key, list value pairs
df = pd.DataFrame(columns = ['Transcription_Factor', 'Target'])
df

In [ ]:
for key, value in d.items():
    lst = value[0]
    for i in range(0, len(lst)):
        term = lst[i]
        df2 = pd.DataFrame([[key, term]], 
                            columns = ['Transcription_Factor', 'Target'])
        df = df.append(df2)

In [ ]:
df

In [ ]:
df.reset_index(drop = True, inplace = True)

df

In [ ]:
df['PubMed_ID'] = ' 15499007'

In [ ]:
#Insert first four columns for .sig file format (NaNs)
df.insert(1, 'NA-1', str(np.nan))
df.insert(2, 'NA-2', str(np.nan))
df.insert(3, 'NA-3', str(np.nan))
df.insert(4, 'NA-4', str(np.nan))

#Insert first four columns for .sig file format (NaNs)
df.insert(6, 'NA-5', str(np.nan))
df.insert(7, 'NA-6', str(np.nan))
df.insert(8, 'NA-7', str(np.nan))
df.insert(9, 'NA-8', str(np.nan))

#Insert column corresponding to sign 
df.insert(10, 'Sign', str(np.nan))

#Insert column specifying interaction type as unknown
df.insert(11, 'Interaction', str(np.nan))

#View dataframe
df.head()

In [ ]:
#Create dictionary 'sigd' with index numbers as keys
sigd = dict([(key, '') for key in df.index])

# loop through rows with iterrows()
for index, rowData in df.iterrows():
    line = ('\t'.join(rowData))
    sigd[index] = line
    
#Transfer tab-separated info into a new txt file
#Make sure to include in ReadMe corresponding column names
with open('ENCODE_2014.sig', 'w') as openfile:
    for index in sigd:
        openfile.write(str(sigd[index]) + '\n')